In [4]:
import os

In [5]:
%pwd

'e:\\work\\DS\\Wine-Quality-Prediction\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'e:\\work\\DS\\Wine-Quality-Prediction'

In [9]:
import dagshub
dagshub.init(repo_owner='XuanHoangee2', repo_name='Wine-Quality-Prediction', mlflow=True)

Accessing as XuanHoangee2

Initialized MLflow to track repo "XuanHoangee2/Wine-Quality-Prediction"

Repository XuanHoangee2/Wine-Quality-Prediction initialized!

In [10]:
import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [12]:
from src.Wine_Quality_Prediction.utils.common import save_json, create_directories, read_yaml
from src.Wine_Quality_Prediction.constants import *

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN.name

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema,
            mlflow_uri= "https://dagshub.com/XuanHoangee2/Wine-Quality-Prediction.mlflow"
        )
        return model_evaluation_config


In [14]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [15]:
class Modelevaluation():
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config

    def evaluation_metric(self, predict, actual):
        rmse = np.sqrt(mean_squared_error(actual ,predict))
        mae = mean_absolute_error(actual, predict)
        r2 = r2_score(actual, predict )

        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme
        if mlflow.active_run():
            mlflow.end_run()

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            (rmse,mae,r2) = self.evaluation_metric(predicted_qualities, test_y)

            score = {"rmse": rmse, "mae": mae, "r2":r2}
            save_json(path= Path(self.config.metric_file_name),data = score)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model,"model", registered_model_name = "ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model,"model")


In [16]:
try:
    config= ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = Modelevaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-12-24 21:43:26,714]: INFO: yaml file: config\config.yaml loaded successfully
[2025-12-24 21:43:26,717]: INFO: yaml file: params.yaml loaded successfully
[2025-12-24 21:43:26,722]: INFO: yaml file: schema.yaml loaded successfully
[2025-12-24 21:43:26,725]: INFO: created directory at: artifacts
[2025-12-24 21:43:26,728]: INFO: created directory at: artifacts/model_evaluation
[2025-12-24 21:43:28,319]: INFO: json file saved at: artifacts\model_evaluation\metrics.json


e:\work\DS\Wine-Quality-Prediction\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/12/24 21:43:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
